# Instruction

**Run each cell one by one**

# Load data

In [1]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pprint, pickle
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id_1_train="13N5Il4V77Od7rK5ywff7YBWDD9PGJBdT"
id_1_evaluation="1RFAezNqq00h8lYs1f_kywJ0lDLhx-sKG"

id_2_train="17YZy5p42Bprj2Bo19lpH04iIBgf8J42l"
id_2_evaluation="13ZuGeuCSopUsPDMUz5TRZfaSuGnA_kZ7"

id_3_train="1mRJkhas7St256PhJPUI4TDQN2ENA2RnO"
id_3_evaluation="1fyk1WmHIk3V_XCDUch28DwWhHXFitogM"

id_4_train="1uEn_rGZMGLEcYuIfrXBcYroAOKTvkVRw"
id_4_evaluation="1jnp1LOgUcZhzcB7weaEQ3FTGk2gNGHcc"


id_5_train="1AGmGhS6pH5ED5v1Qc8RyJbHdybKG4G96"
id_5_evaluation="1IC74vwV5_IyBvBFfSpqIACzgYOI-2i4u"

id_6_train="1Td_zmiqkpKNQBwBje7pyJoRde9wUjABh"
id_6_evaluation="1XiJcjN1-Pe7-R89PugtFspWilF3QOlqi"

id_7_train="1dujTQPIGDBSozlbXt_hJj_qKIftxWVka"
id_7_evaluation="1LniXihHiqx1PSlvmJXu20LaOeXCjBmeQ"


id_8_train="1T66t4HpF1XTB6QNL9ZdwILWGU5sEmf09"
id_8_evaluation="1lxZVBtjtdzF7aZPx4oh2265m-049XgnO"

id_9_train="1zGIpgC3RNRb1qF8t-id3IGtvd92C7IQV"
id_9_evaluation="1guGz6wBBwzTrRNQGCvP-ar2GuI8ZQWe7"

def load(id_train,id_evaluation,subject):
  downloaded = drive.CreateFile({'id':id_train}) 
  downloaded.GetContentFile('B0'+str(subject)+'T.mat')
  downloaded = drive.CreateFile({'id':id_evaluation}) 
  downloaded.GetContentFile('B0'+str(subject)+'E.mat')

load(id_1_train,id_1_evaluation,1)
load(id_2_train,id_2_evaluation,2)
load(id_3_train,id_3_evaluation,3)

load(id_4_train,id_4_evaluation,4)
load(id_5_train,id_5_evaluation,5)
load(id_6_train,id_6_evaluation,6)

load(id_7_train,id_7_evaluation,7)
load(id_8_train,id_8_evaluation,8)
load(id_9_train,id_9_evaluation,9)

# Load Package

In [2]:
import numpy as np
import scipy.io as sio
from tensorflow.keras.utils import to_categorical
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd.function import Function
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader,TensorDataset
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plot
from sklearn.decomposition import PCA
import torch.utils.data as Data
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from torchvision import models
from torchsummary import summary
import torch.optim.lr_scheduler as lr_scheduler


#  Convert the data in mat format into Numpy array

In [3]:
def load_all_data (crossValidation, data_path): 

    big_X_train, big_y_train, big_X_test, big_y_test = [None]*9, [None]*9, [None]*9, [None]*9
    for subject in range (0,9):
        path = data_path+'s' + str(subject+1) + '/'
        big_X_train[subject], big_y_train[subject] = get_data(subject+1, True ,path)
        big_X_test[subject], big_y_test[subject] = get_data(subject+1, False ,path)
    
    return big_X_train, big_y_train, big_X_test, big_y_test

def get_data(subject,training,path, highpass = False):
	'''	Loads the dataset 2a of the BCI Competition IV
	available on http://bnci-horizon-2020.eu/database/data-sets

	Keyword arguments:
	subject -- number of subject in [1, .. ,9]
	training -- if True, load training data
				if False, load testing data
	
	Return:	data_return 	numpy matrix 	size = NO_valid_trial x 22 x 1750
			class_return 	numpy matrix 	size = NO_valid_trial
	'''
	NO_channels = 3
	NO_tests = 120+120+160	
	Window_Length = 7*250 

	class_return = np.zeros(NO_tests)
	data_return = np.zeros((NO_tests,NO_channels,Window_Length))

	NO_valid_trial = 0
	if training:
		a = sio.loadmat(path+'B0'+str(subject)+'T.mat')
	else:
		a = sio.loadmat(path+'B0'+str(subject)+'E.mat')
	a_data = a['data']
	for ii in range(0,a_data.size):
		a_data1 = a_data[0,ii]
		a_data2= [a_data1[0,0]]
		a_data3= a_data2[0]
		a_X 		= a_data3[0]
		a_trial 	= a_data3[1]
		a_y 		= a_data3[2]
		a_fs 		= a_data3[3]
		a_classes 	= a_data3[4]
		a_artifacts = a_data3[5]
		a_gender 	= a_data3[6]
		a_age 		= a_data3[7]

		for trial in range(0,a_trial.size):
			if(a_artifacts[trial]==0):
				data_return[NO_valid_trial,:,:] = np.transpose(a_X[int(a_trial[trial]):(int(a_trial[trial])+Window_Length),:3])
				class_return[NO_valid_trial] = int(a_y[trial])
				NO_valid_trial +=1


	return data_return[0:NO_valid_trial,:,:], class_return[0:NO_valid_trial]

def prepare_features(path,subject,crossValidation=False):
    fs = 250 
    t1 = int(3*fs)
    t2 = int(7*fs)
    T = t2-t1
    X_train, y_train = get_data(subject+1,True,path)
    if crossValidation:
        X_train, X_test, y_train, y_test = train_test_split(
            X_train, y_train, test_size=0.2, random_state=0)
    else:
        X_test, y_test = get_data(subject+1,False,path)

    # prepare training data 	
    N_tr,N_ch,_ =X_train.shape 
    X_train = X_train[:,:,t1:t2].reshape(N_tr,1,N_ch,T)
    y_train_onehot = (y_train-1).astype(int)
    y_train_onehot = to_categorical(y_train_onehot)
    # prepare testing data 
    N_test,N_ch,_ =X_test.shape 
    X_test = X_test[:,:,t1:t2].reshape(N_test,1,N_ch,T)
    y_test_onehot = (y_test-1).astype(int)
    y_test_onehot = to_categorical(y_test_onehot)	
    return X_train,y_train,y_train_onehot,X_test,y_test,y_test_onehot

# Data Load

In [4]:
def train(Subject_number):

#Prepare the raw feature data for training and testing#
  #You can change the "Subject_number" (0-8 corresponding to B01-B09, respectively) to try training and testing between different subjects
  Subject_number=Subject_number
  BATCH_SIZE = 128
  x_train,y_train,y_train_onehot,x_test,y_test,y_test_onehot = prepare_features('',Subject_number,False)
  
  total_x=np.vstack((x_train,x_test))
  total_y=np.hstack((y_train,y_test))
  # total_y_onehot=np.vstack((y_train_onehot,y_test_onehot))
  index1=np.where(total_y == 1)[0]
  index2=np.where(total_y == 2)[0]
  choice1=np.random.choice(index1,10,replace=False)
  choice2=np.random.choice(index2,10,replace=False)
  choice_total=np.hstack((choice1,choice2))
  xxxtest= np.delete(total_x, choice_total,0)
  yyytest=np.delete(total_y, choice_total)


  target_x=total_x[choice_total]
  target_y=total_y[choice_total]



  total_x_train=[]
  total_y_train=[]
  full_subjectnum=[0,1,2,3,4,5,6,7,8]
  full_subjectnum.remove(Subject_number)
  for train_subject in full_subjectnum:
    x_train,y_train,y_train_onehot,x_test,y_test,y_test_onehot = prepare_features('',train_subject,False)

    if len(total_x_train)==0:
      total_x_train=x_train
      total_y_train=y_train
    else:
      total_x_train=np.vstack((total_x_train,x_train))
      total_y_train=np.hstack((total_y_train,y_train))
    total_x_train=np.vstack((total_x_train,x_test))
    total_y_train=np.hstack((total_y_train,y_test))
  x_train=total_x_train
  y_train=total_y_train


  x_test=xxxtest
  y_test=yyytest




  target_x=torch.from_numpy(target_x)
  target_y=torch.from_numpy(target_y-1)
  target_x=target_x.float()
  target_y=target_y.long()




  x_train=torch.from_numpy(x_train)
  y_train=torch.from_numpy(y_train-1)
  x_test=torch.from_numpy(x_test)
  y_test=torch.from_numpy(y_test-1)
  x_train=x_train.float()
  x_test=x_test.float()
  y_train=y_train.long()
  y_test=y_test.long()
  print(x_train.shape)
  source_dataset=Data.TensorDataset(x_train,y_train)
  target_dataset=Data.TensorDataset(target_x,target_y)
  val_dataset=Data.TensorDataset(x_test,y_test)


    
  source_loader= Data.DataLoader(source_dataset, batch_size=400, shuffle=True, num_workers=2)
  target_loader = Data.DataLoader(target_dataset, batch_size=2, shuffle=True, num_workers=2)
  test_loader = Data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)


  return source_loader,target_loader,test_loader

# Network

In [5]:
from torch.autograd import Function


class ReverseLayerF(Function):

    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None


class Constrainedlinear(nn.Linear):
    def _max_norm(self,w,max_val=0.25,eps=1e-8):
        norm = w.norm(2, dim=1, keepdim=True)
        desired = torch.clamp(norm, 0, max_val)
        return w * (desired / (eps + norm))
    def forward(self, input):
        return F.linear(input, self._max_norm(self.weight),self.bias)


In [6]:
import torch.nn as nn


class CNNModel(nn.Module):

    def __init__(self,classes_num):
        super(CNNModel, self).__init__()
        self.numC=3

        self.drop_out = 0.2
        
        self.block_1 = nn.Sequential(
            # Pads the input tensor boundaries with zero
            # left, right, up, bottom
            nn.ZeroPad2d((15, 16, 0, 0)),
            nn.Conv2d(
                in_channels=1,          # input shape (1, C, T)
                out_channels=8,         # num_filters
                kernel_size=(1, 32),    # filter size
                bias=False
            ),                          # output shape (8, C, T)
            nn.BatchNorm2d(8)           # output shape (8, C, T)
        )
        
        # block 2 and 3 are implementations of Depthwise Convolution and Separable Convolution
        self.block_2 = nn.Sequential(
            nn.Conv2d(
                in_channels=8,          # input shape (8, C, T)
                out_channels=16,        # num_filters
                kernel_size=(self.numC, 1),    # filter size
                groups=8,
                bias=False
            ),                          # output shape (16, 1, T)
            nn.BatchNorm2d(16),         # output shape (16, 1, T)
            nn.ELU(),
            nn.AvgPool2d((1, 8)),       # output shape (16, 1, T//4)
            nn.Dropout(self.drop_out)   # output shape (16, 1, T//4)
        )
        
        self.block_3 = nn.Sequential(
            nn.ZeroPad2d((7, 8, 0, 0)),
            nn.Conv2d(
               in_channels=16,          # input shape (16, 1, T//4)
               out_channels=16,         # num_filters
               kernel_size=(1, 16),     # filter size
               groups=16,
               bias=False
            ),                          # output shape (16, 1, T//4)
            nn.Conv2d(
                in_channels=16,         # input shape (16, 1, T//4)
                out_channels=16,        # num_filters
                kernel_size=(1, 1),     # filter size
                bias=False
            ),                          # output shape (16, 1, T//4)
            nn.BatchNorm2d(16),         # output shape (16, 1, T//4)
            nn.ELU(),
            nn.AvgPool2d((1, 8)),       # output shape (16, 1, T//32)
            nn.Dropout(self.drop_out)
        )

        self.class_classifier = nn.Sequential()
        self.class_classifier.add_module('c_fc1',  Constrainedlinear((16 * 15), classes_num))
        self.class_classifier.add_module('c_softmax', nn.LogSoftmax())




        self.domain_classifier = nn.Sequential()
        self.domain_classifier.add_module('d_fc1', nn.Linear((16 * 15), 100))
        self.domain_classifier.add_module('d_bn1', nn.BatchNorm1d(100))
        self.domain_classifier.add_module('d_relu1', nn.ReLU(True))
        self.domain_classifier.add_module('d_fc2', nn.Linear(100, 2))
        self.domain_classifier.add_module('d_softmax', nn.LogSoftmax(dim=1))

    def forward(self, x, alpha):

        x = self.block_1(x)
        x = self.block_2(x)
        x = self.block_3(x)
        
        feature = x.view(x.size(0), -1)
        reverse_feature = ReverseLayerF.apply(feature, alpha)
        class_output = self.class_classifier(feature)
        domain_output = self.domain_classifier(reverse_feature)

        return feature, class_output, domain_output

#Center loss

In [7]:
class CenterLoss(nn.Module):
    def __init__(self, num_classes, feat_dim, size_average=True):
        super(CenterLoss, self).__init__()
        self.centers = nn.Parameter(torch.randn(num_classes, feat_dim))
        self.centerlossfunc = CenterlossFunc.apply
        self.feat_dim = feat_dim
        self.size_average = size_average

    def forward(self, label, feat):
        batch_size = feat.size(0)
        feat = feat.view(batch_size, -1)
        # To check the dim of centers and features
        if feat.size(1) != self.feat_dim:
            raise ValueError("Center's dim: {0} should be equal to input feature's \
                            dim: {1}".format(self.feat_dim,feat.size(1)))
        batch_size_tensor = feat.new_empty(1).fill_(batch_size if self.size_average else 1)
        loss = self.centerlossfunc(feat, label, self.centers, batch_size_tensor)
        return loss


class CenterlossFunc(Function):
    @staticmethod
    def forward(ctx, feature, label, centers, batch_size):
        ctx.save_for_backward(feature, label, centers, batch_size)
        centers_batch = centers.index_select(0, label.long())
        return (feature - centers_batch).pow(2).sum() / 2.0 / batch_size

    @staticmethod
    def backward(ctx, grad_output):
        feature, label, centers, batch_size = ctx.saved_tensors
        centers_batch = centers.index_select(0, label.long())
        diff = centers_batch - feature
        # init every iteration
        counts = centers.new_ones(centers.size(0))
        ones = centers.new_ones(label.size(0))
        grad_centers = centers.new_zeros(centers.size())

        counts = counts.scatter_add_(0, label.long(), ones)
        grad_centers.scatter_add_(0, label.unsqueeze(1).expand(feature.size()).long(), diff)
        grad_centers = grad_centers/counts.view(-1, 1)
        return - grad_output * diff / batch_size, None, grad_centers / batch_size, None

#Testing function
def main(test_cuda=False):
    print('-'*80)
    device = torch.device("cuda" if test_cuda else "cpu")
    ct = CenterLoss(10,2,size_average=True).to(device)
    y = torch.Tensor([0,0,2,1]).to(device)
    feat = torch.zeros(4,2).to(device).requires_grad_()
    print (list(ct.parameters()))
    
    print (ct.centers.grad)
    out = ct(y,feat)
    print(out.item())
    out.backward()


if __name__ == '__main__':
    torch.manual_seed(999)
    main(test_cuda=False)
    if torch.cuda.is_available():
        main(test_cuda=True)

--------------------------------------------------------------------------------
[Parameter containing:
tensor([[-0.2528,  1.4072],
        [ 0.2910,  1.0365],
        [ 0.6396, -1.0857],
        [-1.6153,  1.5635],
        [ 0.1878, -0.9564],
        [-0.2440, -0.4153],
        [ 0.3259, -1.6059],
        [-0.5272,  0.3401],
        [-1.6526,  0.2108],
        [-1.3302,  0.4676]], requires_grad=True)]
None
0.8543729186058044
--------------------------------------------------------------------------------
[Parameter containing:
tensor([[ 0.9147, -1.1896],
        [-0.7501, -1.5465],
        [-0.4303,  0.3703],
        [ 0.4588,  0.2950],
        [ 0.2126, -0.1098],
        [-0.1349,  2.0038],
        [ 1.8339,  2.3544],
        [ 0.3990, -0.3584],
        [-0.2116,  0.5276],
        [ 0.5683, -0.6786]], device='cuda:0', requires_grad=True)]
None
0.9725638628005981


In [8]:
import os
import torch.backends.cudnn as cudnn
import torch.utils.data
from torchvision import transforms
from torchvision import datasets


def test(dataset_name,dataloader, epoch):


    cuda = True
    cudnn.benchmark = True
    alpha = 0



    """ training """
    model_root='/'
    my_net = torch.load(os.path.join(
        model_root, 'mnist_mnistm_model_epoch_' + str(epoch) + '.pth'
    ))
    my_net = my_net.eval()

    if cuda:
        my_net = my_net.cuda()

    len_dataloader = len(dataloader)
    data_target_iter = iter(dataloader)

    i = 0
    n_total = 0
    n_correct = 0

    while i < len_dataloader:

        # test model using target data
        data_target = data_target_iter.next()
        t_img, t_label = data_target

        batch_size = len(t_label)

        input_img = torch.FloatTensor(batch_size, 1, 3, 1000)
        class_label = torch.LongTensor(batch_size)

        if cuda:
            t_img = t_img.cuda()
            t_label = t_label.cuda()
            input_img = input_img.cuda()
            class_label = class_label.cuda()

        input_img.resize_as_(t_img).copy_(t_img)
        class_label.resize_as_(t_label).copy_(t_label)

        _,class_output, _ = my_net(x=input_img, alpha=alpha)
        pred = class_output.data.max(1, keepdim=True)[1]
        n_correct += pred.eq(class_label.data.view_as(pred)).cpu().sum()
        n_total += batch_size

        i += 1

    accu = n_correct.data.numpy() * 1.0 / n_total

    print('epoch: %d, accuracy of the %s dataset: %f' % (epoch, dataset_name, accu))
    return accu

In [11]:
import os
import torch.backends.cudnn as cudnn
import torch.utils.data
from torchvision import transforms
from torchvision import datasets
import random
import os
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
from torchvision import datasets
from torchvision import transforms
import numpy as np

# Change the subject number 0-8 corrsponding to subject B01-B09; Subject_number=0

Subject_number=3
source_loader,target_loader,test_loader=train(Subject_number)


cuda = True
cudnn.benchmark = True
lr = 1e-3
batch_size = 128
n_epoch = 250


manual_seed = random.randint(1, 10000)
random.seed(manual_seed)
torch.manual_seed(manual_seed)

my_net = CNNModel(2)


centerloss = CenterLoss(2, 240).cuda()
optimzer4center = optim.Adam(centerloss.parameters(), lr =0.5)
optimizer = optim.Adam(my_net.parameters(), lr=lr)

loss_class = torch.nn.NLLLoss()
loss_domain = torch.nn.NLLLoss()

if cuda:
    my_net = my_net.cuda()
    loss_class = loss_class.cuda()
    loss_domain = loss_domain.cuda()

for p in my_net.parameters():
    p.requires_grad = True


source_loader,target_loader,test_loader=train(Subject_number)
dataloader_source=source_loader
dataloader_target=target_loader

model_root = os.path.join('')
trainacc=[]
testacc=[]
targacc=[]
for epoch in range(n_epoch):

    len_dataloader = min(len(dataloader_source), len(dataloader_target))
    data_source_iter = iter(dataloader_source)
    data_target_iter = iter(dataloader_target)

    i = 0
    while i < len_dataloader:

        p = float(i + epoch * len_dataloader) / n_epoch / len_dataloader
        alpha = 2. / (1. + np.exp(-10 * p)) - 1

        # training model using source data
        data_source = data_source_iter.next()
        s_img, s_label = data_source




        my_net.zero_grad()
        batch_size = len(s_label)
        input_img = torch.FloatTensor(batch_size, 1, 3, 1000)


        class_label = torch.LongTensor(batch_size)
        domain_label = torch.zeros(batch_size)
        domain_label = domain_label.long()

        if cuda:
            s_img = s_img.cuda()
            s_label = s_label.cuda()
            input_img = input_img.cuda()
            class_label = class_label.cuda()
            domain_label = domain_label.cuda()

        input_img.resize_as_(s_img).copy_(s_img)
        class_label.resize_as_(s_label).copy_(s_label)


        _,class_output, domain_output = my_net(x=input_img, alpha=alpha)

        err_s_label = loss_class(class_output, class_label)
        err_s_domain = loss_domain(domain_output, domain_label)


        data_target = data_target_iter.next()
        t_img, t_label = data_target

        batch_size = len(t_img)
        class_label = torch.LongTensor(batch_size)


        centerloss.zero_grad()

        input_img = torch.FloatTensor(batch_size, 1, 3, 1000)
        domain_label = torch.ones(batch_size)
        domain_label = domain_label.long()

        if cuda:
            t_img = t_img.cuda()
            t_label = t_label.cuda()
            input_img = input_img.cuda()
            class_label = class_label.cuda()
            domain_label = domain_label.cuda()

        input_img.resize_as_(t_img).copy_(t_img)
        class_label.resize_as_(t_label).copy_(t_label)


        fea,class_output, domain_output = my_net(x=input_img, alpha=alpha)

        cens=centerloss(class_label, fea)

        err_t_label = loss_class(class_output, class_label)
        err_t_domain = loss_domain(domain_output, domain_label)
        err = err_t_domain + err_s_domain + 10*err_s_label+err_t_label+0.005*cens

        err.backward()
        optimizer.step()

        # print ('epoch: %d, [iter: %d / all %d], err_s_label: %f, err_s_domain: %f, err_t_domain: %f' \
        #       % (epoch, i, len_dataloader, err_s_label.cpu().data.numpy(),
        #          err_s_domain.cpu().data.numpy(), err_t_domain.cpu().data.numpy()))
    
        i=i+1

    torch.save(my_net, '{0}/mnist_mnistm_model_epoch_{1}.pth'.format(model_root, epoch))
traac=test('train',dataloader_source, epoch)
targac=test('target',dataloader_target, epoch)
tesac=test('test',test_loader, epoch)


torch.Size([4561, 1, 3, 1000])
torch.Size([4561, 1, 3, 1000])


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


epoch: 249, accuracy of the train dataset: 0.744793
epoch: 249, accuracy of the target dataset: 1.000000
epoch: 249, accuracy of the test dataset: 0.870262
